# **KWH Model Report**
### ***Goal**: Build a model that predicts consumption*

Showcase the following:

1. Data Engineering
2. Solution
3. Problem methodology
4. Code
5. Analysis write-up

## **Approach**:
1. Explore and understand the dataset
2. Data Engineering: Cleaning / Feature Engineering / Pipeline
3. Baseline model (Linear Regression)
4. Evaluate, adjust pipeline & test various models
5. Bonus

# **Findings:**



## 1. **EDA**

For this model we will be using the [2009 RESIDENTIAL ENERGY CONSUMPTION SURVEY (RECS) Survey Data](https://www.eia.gov/consumption/residential/data/2009/index.php?view=microdata). This is data collected by the U.S. Energy Information Administration. 

Data Tables:
| File Name                      	| Shape        	| Description                                                   	|
|--------------------------------	|--------------	|---------------------------------------------------------------	|
| [recs2009_public.csv](https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public.csv)            	| (12083, 940) 	| Sample represents 113.6 million U.S. households in 2009       	|
| [public_layout.csv](https://www.eia.gov/consumption/residential/data/2009/csv/public_layout.csv)              	| (940, 5)     	| Descriptive labels and formats for each data variable         	|
| [recs2009_public_repweights.csv](https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public_repweights.csv) 	| (12083, 246) 	| Replicate weights for each of the 12,083 RECS household cases 	|

For the timeline of this mini project I focused on using the first 2 tables. However, for model improvement I would look at utilizing data from the weights table to test for improvements in the model.



### **Inital Data Comments**:
[public table](https://www.eia.gov/consumption/residential/data/2009/csv/recs2009_public.csv) 
![public table](../output/content/data_ex1.jpg)


1. As there are 940 columns we need to be more programatic to how we handle data. Taking the time to review each feature would be a large time investment and in a business would be best to work a SME to expedite understanding. 

2. There is dataleak around the target variable which needed investigation

3. There are many columns with multicoliniarity 

4. Data types should be confirmed for modeling (chategorical, continous, ordinal, nominal etc)

5. What features will be available for prediction? This was probably the hardest to answer, thus working to understand the data was critical for this model.

### **Taget Variable:** 'KWH'

Through exploring the data it was found that there are other columns which are subsets of the target variable. 

<img src="../output/content/target_dropvars.png" width="400">

**Assumption**: We would not have access to this data when making the prediction due to it being dependent on the target variable of KWH. Anything following this logic was droped as part of data processing.

### **-2 Values:** There are a lot of them
There was a lof of occurances of -2 in many columns. The below example shows column names and their count of -2 values. Remembering that theire is 12083 rows in this dataset, many columns contain -2 as most of their values. 

<img src="../output/content/neg_list.png" width="200">


<br>


Most of these columns are data that is only collected in some obersvations. For example, the "AGEHHMEMCAT14" feature indicates if there is a 14th houshold member. Most household memebers do not have 14 people and thus, "-2" is input. 

**Assumption**: when -2 is pressent in large quantities for a feature, it represents a *NULL* value. For the pipeline of this model I removed columns where most of the data was *NULL*

### **Low Unique Value Count:**
In my search to understand the data I was investigating columns that were potentially Ordinal or Nominal. I discovered that many columns are flags as also seen in the "-2" exaple above.  

<img src="../output/content/neg_example1.png" width="500">


### **High Unique Value Count:**
After some basic processing, showing columns with the highest unique value count helped identify additional columns that may need to be investigated.

<img src="../output/content/h_val_count.jpg" width="500">

For example, after droping the '*BTUEL*' column (which was another measure of the target vatriable), I found that 'TOTALBTU' was another column which should have been dropped due to its dependency on knowing the taget variable of KWH.


## 2. **Pipeline**

As it stands in this version of the project:

1. Basic cleaning
2. Encode categorical
3. Drop ID columns
4. Drop target variable dependent columns
5. Remove columns with multicollinearity
6. Remove columns with more than 40% null values.
7. Save transformed data to csv

## 3/4. **Modeling**

### **3. Baseline Model:** Linear Regression (LR)

For my first baseline attempt I had yet to identify columns to drop that had data leak. With only some basic data cleaning and using linear regression the model had a perfect R2 of 1. Too good to be true naturally.

![public table](../output/content/model_b1.jpg)
This alerted me to the fact that there was some major data leakage in the data.

<br>

### **Baseline Model:** (LR) with improved pipeline
After exploring the data further and with some of the discoveries built into my datapipeline a linear regression model started to show more reasonable results with an adjusted R2 on test data of about 90%. 

![public table](../output/content/model_b1.1.jpg)
For this model I did try scale the data based on the train data and then used that scaling model to transform test data for evaluation.

<br>

### **Model Comparisons:**
Using the data in the current format I decided to run a comparison various models [Pycaret](https://pycaret.org/). I have used this AutoML approach as a time saver and sense check that allows for a quick turn around for business value delivery. This high level view better helps guide us for what potential models we should test more extensivly.

![model_compare1](../output/content/model_b1.1.compare.png)

#### Observations of Results:

- Consider at this point in my pipeline that I have not removed outliers... 
- Highlighted in yellow, Lasso Regression making a more generlized model that can combat overfitting. 
- However, a Light Gradient Boosting Machine model has the lowest "***Root Mean Squared Log Error(RMSLE)***".
    - RMSLE has robustness to the effect of the outliers. 

I will test the following models:
- Lasso Regression
- Light Gradient Boosting Machine

### **Lasso Results:**

Some basic tuning did not noticably increase the MAE/MSE/RMSE/R2.... but it did reduce RMSLE which is a potential step in the right direction.

![Lasso Tune](../output/content/model_lasso_tune.jpg)

### **Lightgbm Results:**

Learn more about Lightgbm model [here](https://www.analyticssteps.com/blogs/what-light-gbm-algorithm-how-use-it)
This is a tree based algorithm which is fast but also prone to overfitting if used on less than 10,000 rows. This model worked well for our data. 

Getting almost similar results in our evaluation metrics but having a lower RMSLE this could be a great option to pursue further. 

![lightgbm Tune](../output/content/model_lightgbm1.jpg)

## **5. Bonus:** Still explore

Still experimenting with:

- More columns to remove based on better understanding data
- PCA: number of PCA components would be where eigen values are >= 1
- Outliers: Experiment with outliers threshold and how it impacts the model
- Converting target into BINS to create a classification model
    - Check confusion matrix & kappa value for assesment of performance in the real world

Refactoring:
- Create functions
- Pipeline refactor using Sklearn Pipeline


# Code 
I have squished the code into this 1 notebook for submission

# Housekeeping (Imports / Functions)

In [19]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle

from pycaret.regression import *

In [10]:
def hist(x):
    "returns a widget for histplot"
    return sns.histplot(df1[x], kde=True, stat='density', linewidth=0)

## Data

In [2]:
dpath = '../data/'
df1 = pd.read_csv(dpath + 'recs2009_public.csv')
df_lay = pd.read_csv(dpath + "public_layout.csv")
df_w = pd.read_csv(dpath + "recs2009_public_repweights.csv")

/Users/josephluiz/miniconda3/envs/THIRD_LOVE/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (717,718) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
print(df1.shape)
df1.head()

(12083, 940)


,DOEID,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,...,SCALEKER,IECC_Climate_Pub,HDD50,CDD80,GND_HDD65,WSF,OA_LAT,GWT,DesignDBT99,DesignDBT1
0,1,2,4,12,2,2471.68,4742,1080,4953,1271,...,-2,4A,2117,56,4250,0.48,6,56,9,96
1,2,4,10,26,2,8599.17,2662,199,2688,143,...,-2,3C,62,26,2393,0.61,0,64,38,73
2,3,1,1,1,5,8969.92,6233,505,5741,829,...,-2,5A,2346,49,5654,0.48,3,52,12,88
3,4,2,3,7,2,18003.64,6034,672,5781,868,...,-2,5A,2746,0,4941,0.55,4,55,7,87
4,5,1,1,1,3,5999.61,5388,702,5313,797,...,-2,5A,2251,0,5426,0.61,4,50,13,90


In [8]:
print(df_lay.shape)
df_lay.head()

(940, 5)


,Variable Name,Variable Label,Variable Order in File,Variable Type,Length
0,DOEID,Unique identifier for each respondent,1,Character,5
1,REGIONC,Census Region,2,Numeric,8
2,DIVISION,Census Division,3,Numeric,8
3,REPORTABLE_DOMAIN,Reportable states and groups of states,4,Numeric,8
4,TYPEHUQ,Type of housing unit,5,Numeric,8


In [11]:
# interactive plot for all columns to explore 
interact(hist, x=df1.dtypes[df1.dtypes != 'object'].index)

interactive(children=(Dropdown(description='x', options=('DOEID', 'REGIONC', 'DIVISION', 'REPORTABLE_DOMAIN', …

<function __main__.hist(x)>

In [13]:
# unique values count w descriptions
s_uni_vals = df1.nunique().sort_values()
s_more_than2 = s_uni_vals.where(s_uni_vals > 2).sort_values()
mt2 = s_more_than2.to_frame().reset_index().rename(columns={"index":"Variable Name", 0: "Unique Values Count"})
mt2 = mt2.merge(df_lay[['Variable Name','Variable Label']], on='Variable Name', how='left')
mt2.sort_values(by = "Unique Values Count", ascending=False).head(30)

,Variable Name,Unique Values Count,Variable Label
490,DOEID,12083.0,Unique identifier for each respondent
489,BTUELOTH,12062.0,Electricity usage for other purposes (all end-...
488,KWHOTH,12025.0,Electricity usage for other purposes (all end-...
487,BTUELRFG,11956.0,"Electricity usage for refrigerators, in thousa..."
486,KWHRFG,11711.0,"Electricity usage for refrigerators, in kilowa..."
485,DOLELOTH,11592.0,Electricity cost for other purposes (all end-u...
484,TOTALBTU,11512.0,"Total usage, in thousand BTU, 2009"
483,TOTALBTUOTH,10743.0,"Total usage for appliances, electronics, light..."
482,TOTALBTUSPH,10630.0,"Total usage for space heating, in thousand BTU..."
481,BTUELCOL,9901.0,"Electricity usage for air-conditioning, centra..."


In [14]:
# Columns containing KWH
col_list = df1.columns.to_list()
data = [ 1 if "KWH" in column else 0 for column in col_list]
s_temp = pd.Series(data, index=col_list)
s_temp[s_temp == 1].index.to_list()

['KWH', 'KWHSPH', 'KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH']

## Pipeline

In [12]:
# 0. make now df for cleaned data
df_clean = df1.copy(deep=True)


# 1. clean NOCRCASH, NKRGALNC columns
# by looking at columns with dtype = "object" was able to ID these columns as being non-numeric when they should have
df_clean['NOCRCASH'] = df_clean['NOCRCASH'].replace(".", 0).astype('int64')
df_clean['NKRGALNC'] = df_clean['NKRGALNC'].replace(".", 0).astype('int64')
print("'NOCRCASH', 'NKRGALNC' columns cleaned")

# 2. encode categorical identified columns
# using the "public_layout" file we could identify which features were meant to be "characters"
# ZTOTSQFT columns are already encoded with 0's & 1's so can let them be
# I encoded the remaining categorical columns

encoded_cols = pd.get_dummies(df_clean[['METROMICRO', 'UR', 'IECC_Climate_Pub']])
df_clean.drop(['METROMICRO', 'UR', 'IECC_Climate_Pub'], axis=1, inplace=True)
df_clean = pd.concat([df_clean, encoded_cols], axis = 1)
print("'METROMICRO', 'UR', 'IECC_Climate_Pub' columns encoded")

# 3. Remove the DOEID column as that is the unique ID for each record
df_clean.drop(['DOEID'], axis=1, inplace=True)
print("Dropped 'DOEID' column")

# 4. Remove columns depended on knowing the target variable of KWH
# BTUEL has 1:1 correlation with target and is anothe measurement of the target variable
df_clean.drop(['BTUEL'], axis=1, inplace=True)
print("Assumption: BTUEL values are only known when knowing the target variable...")
print("Dropped 'BTUEL' column")

# There are columns that are components of the total KWH consumption
kwh_remove = ['KWHSPH', 'KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH', 'TOTALBTU']
df_clean.drop(kwh_remove, axis=1, inplace=True)
print("Dropped 'KWH composite columns' column")

# 5. Remove columns with multicollinearity
print("Processing correlation...")
df_cor = df_clean.corr().abs()
# Select upper triangle of correlation matrix
upper = df_cor.where(np.triu(np.ones(df_cor.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]
print("Reducing multicollinearity by", len(to_drop), "columns")
df_clean.drop(to_drop, axis=1, inplace=True)

# 6. Remove columns with more than 40% null values.
# Assumption: where -2 is pressent that this is a null value 
df_count2 = df_clean.isin([-2]).sum(axis=0).sort_values(ascending=False)
percent2_thresh = len(df_clean) * 0.4
df_count2.where(df_count2 > percent2_thresh, inplace=True)
df_count2.dropna(how="any", inplace=True)

drop2_list = df_count2.index.to_list()
print("Dropping", len(drop2_list), "columns with > 40% null values")
df_clean.drop(drop2_list, axis=1, inplace=True) 


# Save output to data location
print("Final table shape is:", df_clean.shape)
df_clean.to_csv(dpath + 'pipeline_output.csv', index=False)
print("Saved to data path:" + dpath)

'NOCRCASH', 'NKRGALNC' columns cleaned
'METROMICRO', 'UR', 'IECC_Climate_Pub' columns encoded
Dropped 'DOEID' column
Assumption: BTUEL values are only known when knowing the target variable...
Dropped 'BTUEL' column
Dropped 'KWH composite columns' column
Processing correlation...
Reducing multicollinearity by 353 columns
Dropping 106 columns with > 40% null values
Final table shape is: (12083, 486)
Saved to data path:../data/


## Modeling

In [15]:
# importing data again if from saved location
dpath = dpath # set this to the directory of your data location if diffent to earlier in notebook
df = pd.read_csv(dpath + 'pipeline_output.csv')


In [16]:
# Basic LR w scaling
X = np.array(df.drop('KWH', axis=1))
y = np.array(df['KWH'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=150)

# fit scale on X_train data
sc = StandardScaler()
sc.fit(X_train)
X_train_scaled = sc.transform(X_train)
X_test_scaled = sc.transform(X_test)

# Linear Regression
reg = LinearRegression()
reg.fit(X_train, y_train)

# print scores
print("Train R2:", reg.score(X_train, y_train))
print("Test R2:", reg.score(X_test, y_test))

# display adjusted R-squared
adj_r2_train = 1 - (1-reg.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
adj_r2_test = 1 - (1-reg.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

print("Train Adjusted R2:", adj_r2_train)
print("Test Adjusted R2:", adj_r2_test)


Train R2: 0.9297152640485217
Test R2: 0.9125234373754924
Train Adjusted R2: 0.9254392860083226
Test Adjusted R2: 0.8990076256925086


In [18]:
# save model 
filename = '../output/models/baseline_linreg.sav'
pickle.dump(reg, open(filename, 'wb'))

In [20]:
### pycarret compare
s = setup(data = df, target='KWH', n_jobs=-1)

,Description,Value
0,session_id,3177
1,Target,KWH
2,Original Data,"(12083, 486)"
3,Missing Values,False
4,Numeric Features,24
5,Categorical Features,461
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(8458, 876)"


In [ ]:
best = compare_models()

In [ ]:
# careful as this can be coputationally expensive
# but it will allow you to click around and see dif visuals / stats on your model
evaluate_model(best)

In [ ]:
tuned_lasso = tune_model(best)

In [ ]:
# lets take a look at what the actuall prediction looks like using this model
predict_model(tuned_lasso)

In [ ]:
tuned_lasso.get_params()

In [ ]:
save_model(tuned_lasso, '../output/models/lasso1')

In [ ]:
### Lightgbm Model
#s = setup(data = df, target='KWH', n_jobs=-1, train_size = 0.7, session_id = 123)
s = setup(data = df, target='KWH', n_jobs=-1)

In [ ]:
lightgbm_model = create_model('lightgbm', fold = 10)

In [ ]:
tuned_lightgbm = tune_model(lightgbm_model, optimize= "RMSE")
save_model(tuned_lightgbm, '../output/models/lightgbm1')